In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import stan
import arviz as az

import nest_asyncio
nest_asyncio.apply()

/tmp/ipykernel_94/3982279640.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [10]:
data = pd.read_csv("https://raw.githubusercontent.com/stedy/Machine-Learning-with-R-datasets/master/insurance.csv", \
                    header=0, usecols=['age', 'bmi', 'children', 'charges'])
data

,age,bmi,children,charges
0,19,27.900,0,16884.92400
1,18,33.770,1,1725.55230
2,28,33.000,3,4449.46200
3,33,22.705,0,21984.47061
4,32,28.880,0,3866.85520
...,...,...,...,...
1333,50,30.970,3,10600.54830
1334,18,31.920,0,2205.98080
1335,18,36.850,0,1629.83350
1336,21,25.800,0,2007.94500


In [12]:
# Standardize Predictors
data['bmi'] = (data['bmi'] - data['bmi'].mean()) / data['bmi'].std()
data['age'] = (data['age'] - data['age'].mean()) / data['age'].std()
data['children'] = (data['children'] - data['children'].mean()) / data['children'].std()
data['charges'] = (data['charges'] - data['charges'].mean()) / data['charges'].std()
data


,age,bmi,children,charges
0,-1.438227,-0.453151,-0.908274,0.298472
1,-1.509401,0.509431,-0.078738,-0.953333
2,-0.797655,0.383164,1.580335,-0.728402
3,-0.441782,-1.305043,-0.908274,0.719574
4,-0.512957,-0.292447,-0.908274,-0.776512
...,...,...,...,...
1333,0.768185,0.050278,1.580335,-0.220468
1334,-1.509401,0.206062,-0.908274,-0.913661
1335,-1.509401,1.014499,-0.908274,-0.961237
1336,-1.295877,-0.797515,-0.908274,-0.930014


In [13]:
# Prepare data for Stan
N = len(data)
freqs = np.column_stack((np.ones(N), data['bmi'], data['age'], data['children']))
y = data['charges']
K = freqs.shape[1]

stan_data = {
    'N': N,
    'K': K,
    'freqs': freqs,
    'y': y,
    'tau_0': 1,
    'tau_1': 1,
    'sigma_alpha': 1,
    'sigma_beta': 1
}

In [23]:
vectorized_program_code = """
data {
    int<lower=0> N;         
    int<lower=1> K; 
    array[K] int<lower=0, upper=N> freqs;
    vector[N] y;
    
    real<lower=0> tau_0; 
    real<lower=0> tau_1; 
    real<lower=0> sigma_alpha; 
    real<lower=0> sigma_beta;        
}

parameters {
    real alpha; 
    vector[K] beta; 
    real<lower=0> sigma; 
}

transformed parameters {
    beta[1] ~  ;
    beta[2] ~  ;
    beta[3] ~  ;
}

model {
    // Priors
    sigma ~ inv_gamma(tau_0, tau_1);
    alpha ~ normal(0, sigma_alpha);
    beta ~ multi_normal(0, sigma_beta);
    
    // Likelihood
    y ~ normal(freqs * beta + alpha, sigma);
}

generated quantities{

}
"""

In [24]:
vectorized_posterior = stan.build(vectorized_program_code, data=stan_data)

TypeError: Object of type Series is not JSON serializable

In [ ]:
%%time
fit_vectorized = vectorized_posterior.sample(num_chains=4, num_samples=2000, num_warmup=1000)

In [ ]:
summary = az.summary(fit_vectorized)
summary